In [2]:
import os

In [3]:
%pwd

'd:\\Projects\\Text-summarizer-project\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\Projects\\Text-summarizer-project'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [8]:
from textsummarizer.constants import *
from textsummarizer.utils.common import read_yaml, create_directories

In [15]:
class ConfigurationManager:
    def __init__(
            self, 
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir,
        )

        return data_ingestion_config



In [16]:
import os
import urllib.request as request
import zipfile
from textsummarizer.logging import logger
from textsummarizer.utils.common import get_size


In [17]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
               url= self.config.source_URL, 
               filename=self.config.local_data_file)
            logger.info(f"{filename} download! with following info: \n{headers}")

        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    
    def extract_zip_file(self):
        """
        zip file path: str
        Extracts the zip file to the specified directory.
        Function returns none
        
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [20]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e 

[2025-09-21 16:53:30,770 : INFO: common : yaml file: config\config.yaml loaded successfully]
[2025-09-21 16:53:30,791 : INFO: common : yaml file: param.yaml loaded successfully]
[2025-09-21 16:53:30,804 : INFO: common : created directory at : artifacts]
[2025-09-21 16:53:30,814 : INFO: common : created directory at : artifacts/data_ingestion]


[2025-09-21 16:53:34,858 : INFO: 877528339 : artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 8C31:351B60:1DEAE7:4AC462:68CFE036
Accept-Ranges: bytes
Date: Sun, 21 Sep 2025 11:23:36 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10232-MAA
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1758453815.160244,VS0,VE925
Vary: Authorization,Accept-Encoding
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 63e54ef5f4ab76a1f963b7fba605925cfe326475
Expires: Sun, 21 Sep 2025 11:28:36 GMT
Source-Age: 1

]
